In [3]:
import keras 
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import imutils
import cv2
import PIL.Image
import os
%matplotlib inline

demo_datagen = ImageDataGenerator(
    rotation_range=15,
    width_shift_range=0.05,
    height_shift_range=0.05,
    rescale=1./255,
    shear_range=0.05,
    brightness_range=[0.1, 1.5],
    horizontal_flip=True,
    vertical_flip=True
)
     

In [4]:
def mkdir():
  os.mkdir('testfolder')
  #os.mkdir('testfolder1')

  os.mkdir('testfolder_output')
  #os.mkdir('testfolder_output1')
mkdir()  

In [18]:
def cropAndAugmentation():
    # Initialize augmentation parameters
    demo_datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    IMG_SIZE = 224
    dim = (IMG_SIZE, IMG_SIZE)
    directory = ['Test-no_tumor']  # Input folders
    directory_output = ['Test-no_tumor-output']  # Corresponding output folders

    # Iterate over input-output directory pairs
    for input_folder, output_folder in zip(directory, directory_output):
        j = 0
        for img in os.listdir(f'{input_folder}/'):
            image_path = f'{input_folder}/{img}'
            image = cv2.imread(image_path)

            if image is None:
                print(f"Error reading image: {image_path}")
                continue

            # Resize and process the image
            image = cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
            gray = cv2.cvtColor(image, cv2.COLOR_RGB2GRAY)
            gray = cv2.GaussianBlur(gray, (5, 5), 0)

            # Thresholding and contour detection
            thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
            thresh = cv2.erode(thresh, None, iterations=2)
            thresh = cv2.dilate(thresh, None, iterations=2)

            cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = imutils.grab_contours(cnts)

            if len(cnts) == 0:
                print(f"No contours found in image: {img}")
                continue

            c = max(cnts, key=cv2.contourArea)

            # Get extreme points and crop the image
            extLeft = tuple(c[c[:, :, 0].argmin()][0])
            extRight = tuple(c[c[:, :, 0].argmax()][0])
            extTop = tuple(c[c[:, :, 1].argmin()][0])
            extBot = tuple(c[c[:, :, 1].argmax()][0])

            ADD_PIXELS = 0  # Optional margin around the crop
            new_image = image[extTop[1]-ADD_PIXELS:extBot[1]+ADD_PIXELS, extLeft[0]-ADD_PIXELS:extRight[0]+ADD_PIXELS].copy()

            if new_image.size == 0:
                print(f"Invalid crop for image: {img}")
                continue

            # Augment the cropped image
            x = new_image.reshape((1,) + new_image.shape)
            i = 0

            # Save augmented images to output folder
            for batch in demo_datagen.flow(x, batch_size=1, save_to_dir=output_folder, save_prefix=f'{input_folder}_{j}', save_format='jpg'):
                i += 1
                if i > 20:
                    break
            j += 1

cropAndAugmentation()